<a href="https://colab.research.google.com/github/ignavf/IgnacioValdes/blob/main/Asistente_AFP_MODELO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Asistente de AFP Modelo **Prototipo Digital**

**Instrucciones de Uso**

---



> Inciar el codigo con el boton play de colab, luego utilizar para revisar data, puede visitar el siguiente link para ver la data/dashboard [Link a la data online](https://docs.google.com/spreadsheets/d/1VqwpwBNSsIooVbaqopR721Q0o8g5lDhvjtCQeip3Izw/edit?usp=sharing)




In [19]:
# Modificación del sistema AFP para usar la hoja Calculadora existente
# y permitir cálculos para todos los fondos

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from datetime import datetime, timedelta
import re
import random
import json
import time
import io
import base64
import matplotlib.pyplot as plt
import numpy as np

class DBManager:
    def __init__(self, credentials_file, spreadsheet_name):
        try:
            # Autenticar
            scope = ["https://spreadsheets.google.com/feeds",
                    "https://www.googleapis.com/auth/spreadsheets",
                    "https://www.googleapis.com/auth/drive.file",
                    "https://www.googleapis.com/auth/drive"]

            credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
            self.client = gspread.authorize(credentials)

            # Abrir o crear hoja de cálculo
            try:
                self.spreadsheet = self.client.open(spreadsheet_name)
                print(f"✅ Conectado a '{spreadsheet_name}'")
            except:
                self.spreadsheet = self.client.create(spreadsheet_name)
                print(f"✅ Creada nueva hoja '{spreadsheet_name}'")

            # Verificar que exista la hoja Calculadora
            self._verify_calculadora_sheet()

            # Crear otras hojas necesarias
            self._create_sheets_if_not_exist()

        except Exception as e:
            print(f"❌ Error en la conexión: {e}")
            self.client = None
            self.spreadsheet = None

    def _verify_calculadora_sheet(self):
        """Verifica que exista la hoja Calculadora con las columnas requeridas"""
        existing_sheets = [sheet.title for sheet in self.spreadsheet.worksheets()]

        if "Calculadora" not in existing_sheets:
            print("❌ La hoja 'Calculadora' no existe en el spreadsheet.")
            print("Creando hoja 'Calculadora'...")
            sheet = self.spreadsheet.add_worksheet(title="Calculadora", rows=1000, cols=16)

            # Establecer los encabezados requeridos
            encabezados = [
                "Fecha", "Hora", "Rut", "Nombre", "Edad", "Sexo", "Saldo actual",
                "Sueldo Mensual", "Años cotizando", "Tipo Trabajador", "Mejor fondo",
                "Pension proyectada", "Tasa reemplazo", "Saldo Proyectado",
                "Años Jubilar", "APV Proyectado"
            ]
            sheet.update([encabezados])

            # Formatear la primera fila (encabezados)
            fmt = {
                "backgroundColor": {
                    "red": 0.2,
                    "green": 0.2,
                    "blue": 0.4
                },
                "textFormat": {
                    "foregroundColor": {
                        "red": 1.0,
                        "green": 1.0,
                        "blue": 1.0
                    },
                    "bold": True
                }
            }
            sheet.format("A1:P1", fmt)
            print("✅ Hoja 'Calculadora' creada con éxito")
        else:
            # Verificar que tenga los encabezados correctos
            calculadora_sheet = self.spreadsheet.worksheet("Calculadora")
            headers = calculadora_sheet.row_values(1)
            expected_headers = [
                "Fecha", "Hora", "Rut", "Nombre", "Edad", "Sexo", "Saldo actual",
                "Sueldo Mensual", "Años cotizando", "Tipo Trabajador", "Mejor fondo",
                "Pension proyectada", "Tasa reemplazo", "Saldo Proyectado",
                "Años Jubilar", "APV Proyectado"
            ]

            # Verificar si los encabezados coinciden (ignorando mayúsculas/minúsculas)
            headers_match = all(
                h1.lower() == h2.lower()
                for h1, h2 in zip(headers, expected_headers)
                if headers and h1 and h2
            )

            if not headers_match:
                print("⚠️ Los encabezados de la hoja 'Calculadora' no coinciden con los esperados.")
                print("Se utilizará la estructura existente. Asegúrate de que las columnas coincidan con el uso esperado.")

    def _create_sheets_if_not_exist(self):
        """Crea las hojas necesarias si no existen"""
        existing_sheets = [sheet.title for sheet in self.spreadsheet.worksheets()]

        # Crear hoja Usuarios si no existe
        if "Usuarios" not in existing_sheets:
            print("Creando hoja 'Usuarios'...")
            sheet = self.spreadsheet.add_worksheet(title="Usuarios", rows=1000, cols=12)
            sheet.update([
                ["RUT", "Nombre", "Email", "Teléfono", "Fecha Nacimiento", "Tipo Trabajador",
                "Fecha Registro", "Sueldo Mensual", "Fondo Actual", "Saldo", "Última Cotización", "Género"]
            ])

        # Crear hoja Interacciones si no existe
        if "Interacciones" not in existing_sheets:
            print("Creando hoja 'Interacciones'...")
            sheet = self.spreadsheet.add_worksheet(title="Interacciones", rows=1000, cols=8)
            sheet.update([
                ["RUT", "Fecha Interacción", "Tipo Consulta", "Mensaje Usuario", "Respuesta Bot",
                "Canal", "Duración (seg)", "Satisfacción"]
            ])

        # Crear hoja Métricas si no existe
        if "Métricas" not in existing_sheets:
            print("Creando hoja 'Métricas'...")
            sheet = self.spreadsheet.add_worksheet(title="Métricas", rows=100, cols=6)
            sheet.update([
                ["Fecha", "Total Interacciones", "Nuevos Usuarios", "Consultas Saldo",
                "Consultas Rentabilidad", "Consultas Otros"]
            ])

    def obtener_datos_usuario(self, rut):
        """Obtiene los datos de un usuario desde la hoja de Usuarios"""
        if not self.spreadsheet:
            print("❌ No hay conexión con la base de datos")
            return None

        try:
            # Obtener hoja de usuarios
            usuarios_sheet = self.spreadsheet.worksheet("Usuarios")

            # Buscar RUT en la primera columna
            try:
                cell_list = usuarios_sheet.findall(rut)
                if len(cell_list) > 0:
                    # Encontramos al usuario
                    row = cell_list[0].row
                    row_values = usuarios_sheet.row_values(row)

                    # Convertir valores a sus tipos adecuados
                    # Los índices corresponden a las columnas de la hoja
                    datos_usuario = {
                        "rut": row_values[0],
                        "nombre": row_values[1],
                        "email": row_values[2] if len(row_values) > 2 else "",
                        "telefono": row_values[3] if len(row_values) > 3 else "",
                        "fecha_nacimiento": row_values[4] if len(row_values) > 4 else "",
                        "tipo_trabajador": row_values[5] if len(row_values) > 5 else "Dependiente",
                        "fecha_registro": row_values[6] if len(row_values) > 6 else "",
                        "sueldo_mensual": self._parse_float(row_values[7]) if len(row_values) > 7 else 0,
                        "fondo": row_values[8] if len(row_values) > 8 else "C",
                        "saldo": self._parse_float(row_values[9]) if len(row_values) > 9 else 0,
                        "ultima_cotizacion": row_values[10] if len(row_values) > 10 else "",
                        "genero": row_values[11].lower() if len(row_values) > 11 and row_values[11] else "hombre"
                    }

                    # Calcular edad si tenemos fecha de nacimiento
                    if datos_usuario["fecha_nacimiento"]:
                        datos_usuario["edad"] = self._calcular_edad(datos_usuario["fecha_nacimiento"])
                    else:
                        datos_usuario["edad"] = 30  # Valor por defecto

                    # Calcular años cotizando (si no se tiene, se estima como edad - 25)
                    datos_usuario["años_cotizando"] = max(0, datos_usuario["edad"] - 25)

                    return datos_usuario
                else:
                    print(f"❌ No se encontró usuario con RUT {rut}")
                    return None
            except Exception as e:
                print(f"❌ Error al buscar usuario: {e}")
                return None

        except Exception as e:
            print(f"❌ Error al obtener datos de usuario: {e}")
            return None

    def guardar_proyeccion_calculadora(self, datos_proyeccion):
        """Guarda la proyección de pensión en la hoja Calculadora"""
        if not self.spreadsheet:
            print("❌ No hay conexión con la base de datos")
            return False

        try:
            # Obtener hoja de Calculadora
            calculadora_sheet = self.spreadsheet.worksheet("Calculadora")

            # Obtener fecha y hora actuales
            ahora = datetime.now()
            fecha = ahora.strftime("%d/%m/%Y")
            hora = ahora.strftime("%H:%M:%S")

            # Preparar datos para guardar
            nueva_fila = [
                fecha,                                         # Fecha
                hora,                                          # Hora
                datos_proyeccion.get("rut", ""),               # Rut
                datos_proyeccion.get("nombre", ""),            # Nombre
                datos_proyeccion.get("edad", ""),              # Edad
                datos_proyeccion.get("genero", ""),            # Sexo
                datos_proyeccion.get("saldo_actual", ""),      # Saldo actual
                datos_proyeccion.get("sueldo_mensual", ""),    # Sueldo Mensual
                datos_proyeccion.get("años_cotizando", ""),    # Años cotizando
                datos_proyeccion.get("tipo_trabajador", ""),   # Tipo Trabajador
                datos_proyeccion.get("mejor_fondo", ""),       # Mejor fondo
                datos_proyeccion.get("pension_mensual", ""),   # Pension proyectada
                datos_proyeccion.get("tasa_reemplazo", ""),    # Tasa reemplazo
                datos_proyeccion.get("saldo_proyectado", ""),  # Saldo Proyectado
                datos_proyeccion.get("años_para_jubilar", ""), # Años Jubilar
                datos_proyeccion.get("apv_proyectado", "")     # APV Proyectado
            ]

            # Guardar en la hoja
            calculadora_sheet.append_row(nueva_fila)
            print(f"✅ Proyección guardada en Calculadora para {datos_proyeccion.get('rut', 'usuario')}")
            return True

        except Exception as e:
            print(f"❌ Error al guardar proyección en Calculadora: {e}")
            return False

    def guardar_proyecciones_por_fondos_en_calculadora(self, datos_usuario, resultados_por_fondo):
        """
        Guarda los resultados de la simulación para cada fondo en la hoja Calculadora
        """
        if not self.spreadsheet:
            print("❌ No hay conexión con la base de datos")
            return False

        try:
            # Determinar mejor fondo (el que da mejor pensión mensual)
            mejor_pension = 0
            mejor_fondo = None

            for fondo, resultado in resultados_por_fondo.items():
                pension = resultado.get('pension_mensual_estimada', 0)
                if pension > mejor_pension:
                    mejor_pension = pension
                    mejor_fondo = fondo

            if not mejor_fondo:
                print("❌ No se pudo determinar el mejor fondo")
                return False

            # Obtener resultado del mejor fondo
            mejor_resultado = resultados_por_fondo[mejor_fondo]

            # Calcular APV recomendado
            apv_mensual = 0
            años_para_jubilar = mejor_resultado.get('años_para_jubilar', 0)
            if años_para_jubilar > 0:
                pension_actual = mejor_resultado.get('pension_mensual_estimada', 0)
                ultimo_sueldo = mejor_resultado.get('ultimo_sueldo_proyectado', 0)
                tasa_actual = mejor_resultado.get('tasa_reemplazo', 0)

                if tasa_actual > 0 and ultimo_sueldo > 0:
                    pension_objetivo = (tasa_actual + 10) / 100 * ultimo_sueldo
                    diferencia_pension = pension_objetivo - pension_actual

                    if diferencia_pension > 0:
                        factor_actuarial = mejor_resultado.get("factor_actuarial", 20)
                        apv_mensual = (diferencia_pension * 12 * factor_actuarial) / (años_para_jubilar * 12)
                        apv_mensual = max(min(apv_mensual, datos_usuario.get("sueldo_mensual", 0) * 0.2),
                                        datos_usuario.get("sueldo_mensual", 0) * 0.05)

            # Preparar datos para guardar en Calculadora
            datos_proyeccion = {
                "rut": datos_usuario.get("rut", ""),
                "nombre": datos_usuario.get("nombre", ""),
                "edad": datos_usuario.get("edad", ""),
                "genero": datos_usuario.get("genero", ""),
                "saldo_actual": datos_usuario.get("saldo_actual", ""),
                "sueldo_mensual": datos_usuario.get("sueldo_mensual", ""),
                "años_cotizando": datos_usuario.get("años_cotizando", ""),
                "tipo_trabajador": datos_usuario.get("tipo_trabajador", ""),
                "mejor_fondo": mejor_fondo,
                "pension_mensual": mejor_resultado.get("pension_mensual_estimada", ""),
                "tasa_reemplazo": mejor_resultado.get("tasa_reemplazo", ""),
                "saldo_proyectado": mejor_resultado.get("saldo_proyectado", ""),
                "años_para_jubilar": mejor_resultado.get("años_para_jubilar", ""),
                "apv_proyectado": apv_mensual
            }

            # Guardar en la hoja Calculadora
            return self.guardar_proyeccion_calculadora(datos_proyeccion)

        except Exception as e:
            print(f"❌ Error al guardar proyecciones por fondos en Calculadora: {e}")
            return False

    def registrar_usuario(self, datos):
        """Registra o actualiza un usuario en la hoja de Usuarios"""
        if not self.spreadsheet:
            print("❌ No hay conexión con la base de datos")
            return False

        try:
            # Obtener hoja de usuarios
            usuarios_sheet = self.spreadsheet.worksheet("Usuarios")

            # Verificar si ya existe el usuario
            rut = datos.get("rut", "")
            if not rut:
                print("❌ No se proporcionó RUT")
                return False

            # Buscar RUT en la primera columna
            cell_list = usuarios_sheet.findall(rut)
            user_exists = len(cell_list) > 0

            # Determinar género basado en nombre (simplificado)
            nombre = datos.get("nombre", "")
            genero = datos.get("genero", "")
            if not genero:
                genero = "mujer" if nombre.split()[-1].lower().endswith(('a', 'z')) else "hombre"

            if user_exists:
                # Obtener número de fila del usuario existente
                row = cell_list[0].row

                # Obtener valores actuales
                row_values = usuarios_sheet.row_values(row)

                # Preparar datos actualizados
                updated_values = [
                    datos.get("rut", ""),
                    datos.get("nombre", ""),
                    datos.get("email", ""),
                    datos.get("telefono", ""),
                    datos.get("fecha_nacimiento", ""),
                    datos.get("tipo_trabajador", ""),
                    row_values[6] if len(row_values) > 6 else datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    datos.get("sueldo_mensual", ""),
                    datos.get("fondo", ""),
                    datos.get("saldo", ""),
                    datos.get("ultima_cotizacion", ""),
                    genero  # Añadimos el género
                ]

                # Actualizar fila
                usuarios_sheet.update(f'A{row}:L{row}', [updated_values])
                print(f"✅ Usuario {rut} actualizado")

            else:
                # Preparar datos del nuevo usuario
                new_user = [
                    datos.get("rut", ""),
                    datos.get("nombre", ""),
                    datos.get("email", ""),
                    datos.get("telefono", ""),
                    datos.get("fecha_nacimiento", ""),
                    datos.get("tipo_trabajador", ""),
                    datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    datos.get("sueldo_mensual", ""),
                    datos.get("fondo", "C"),  # Fondo C por defecto
                    datos.get("saldo", "10000000"),  # Saldo por defecto
                    datos.get("ultima_cotizacion", datetime.now().strftime("%Y-%m-%d")),
                    genero  # Añadimos el género
                ]

                # Añadir nuevo usuario
                usuarios_sheet.append_row(new_user)
                print(f"✅ Nuevo usuario {rut} registrado")

                # Incrementar contador de nuevos usuarios en métricas
                self._actualizar_metrica_nuevos_usuarios()

            return True

        except Exception as e:
            print(f"❌ Error al registrar usuario: {e}")
            return False

    def registrar_interaccion(self, datos):
        """Registra una nueva interacción en la hoja de Interacciones"""
        if not self.spreadsheet:
            print("❌ No hay conexión con la base de datos")
            return False

        try:
            # Obtener hoja de interacciones
            interacciones_sheet = self.spreadsheet.worksheet("Interacciones")

            # Preparar datos
            interaccion = [
                datos.get("rut", "Anónimo"),
                datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                datos.get("tipo_consulta", "General"),
                datos.get("mensaje", ""),
                datos.get("respuesta", ""),
                datos.get("canal", "Web"),
                datos.get("duracion", "0"),
                datos.get("satisfaccion", "")
            ]

            # Registrar interacción
            interacciones_sheet.append_row(interaccion)
            print(f"✅ Interacción registrada para {datos.get('rut', 'anónimo')}")

            # Actualizar métricas
            self._actualizar_metrica_interacciones(datos.get("tipo_consulta", "General"))

            return True

        except Exception as e:
            print(f"❌ Error al registrar interacción: {e}")
            return False

    def _actualizar_metrica_nuevos_usuarios(self):
        """Actualiza el contador de nuevos usuarios en la hoja de Métricas"""
        try:
            metricas_sheet = self.spreadsheet.worksheet("Métricas")
            fecha_hoy = datetime.now().strftime("%Y-%m-%d")

            # Buscar fila con la fecha de hoy
            try:
                celdas = metricas_sheet.findall(fecha_hoy)
                if celdas:
                    # Existe una entrada para hoy
                    fila = celdas[0].row
                    valor_actual = metricas_sheet.cell(fila, 3).value
                    nuevo_valor = int(valor_actual or 0) + 1
                    metricas_sheet.update_cell(fila, 3, nuevo_valor)
                else:
                    # No existe entrada para hoy
                    metricas_sheet.append_row([fecha_hoy, 0, 1, 0, 0, 0])
            except:
                # Si hay error, intentar crear una nueva fila
                metricas_sheet.append_row([fecha_hoy, 0, 1, 0, 0, 0])

        except Exception as e:
            print(f"⚠️ Error al actualizar métrica de nuevos usuarios: {e}")

    def _actualizar_metrica_interacciones(self, tipo_consulta):
        """Actualiza las métricas de interacciones"""
        try:
            metricas_sheet = self.spreadsheet.worksheet("Métricas")
            fecha_hoy = datetime.now().strftime("%Y-%m-%d")

            # Determinar columna a actualizar según tipo de consulta
            columna = 6  # Default: Otros
            if "saldo" in tipo_consulta.lower():
                columna = 4  # Consultas Saldo
            elif "rentabilidad" in tipo_consulta.lower():
                columna = 5  # Consultas Rentabilidad

            # Buscar fila con la fecha de hoy
            try:
                celdas = metricas_sheet.findall(fecha_hoy)
                if celdas:
                    # Existe una entrada para hoy
                    fila = celdas[0].row

                    # Actualizar total de interacciones
                    valor_total = metricas_sheet.cell(fila, 2).value
                    nuevo_total = int(valor_total or 0) + 1
                    metricas_sheet.update_cell(fila, 2, nuevo_total)

                    # Actualizar contador específico del tipo de consulta
                    valor_tipo = metricas_sheet.cell(fila, columna).value
                    nuevo_valor_tipo = int(valor_tipo or 0) + 1
                    metricas_sheet.update_cell(fila, columna, nuevo_valor_tipo)
                else:
                    # No existe entrada para hoy
                    nueva_fila = [fecha_hoy, 1, 0, 0, 0, 0]
                    nueva_fila[columna-1] = 1  # -1 por el índice 0-based de Python
                    metricas_sheet.append_row(nueva_fila)
            except:
                # Si hay error, intentar crear una nueva fila
                nueva_fila = [fecha_hoy, 1, 0, 0, 0, 0]
                nueva_fila[columna-1] = 1
                metricas_sheet.append_row(nueva_fila)

        except Exception as e:
            print(f"⚠️ Error al actualizar métrica de interacciones: {e}")

    def _parse_float(self, value):
        """Convierte un valor a float, maneja valores vacíos y formatos"""
        if not value:
            return 0

        try:
            # Eliminar caracteres no numéricos excepto punto decimal
            clean_value = re.sub(r'[^\d.]', '', str(value))
            return float(clean_value)
        except:
            return 0

    def _calcular_edad(self, fecha_nacimiento):
        """Calcula edad a partir de fecha de nacimiento"""
        try:
            # Intentar diferentes formatos de fecha
            for fmt in ["%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y", "%d-%m-%Y"]:
                try:
                    fecha_nac = datetime.strptime(fecha_nacimiento, fmt)
                    hoy = datetime.now()
                    return hoy.year - fecha_nac.year - ((hoy.month, hoy.day) < (fecha_nac.month, fecha_nac.day))
                except:
                    continue

            # Si no se pudo parsear con ningún formato, intentar extraer solo el año
            if len(fecha_nacimiento) >= 4:
                try:
                    año = int(fecha_nacimiento[:4])
                    return datetime.now().year - año
                except:
                    pass

            return 30  # Valor por defecto en caso de error
        except:
            return 30  # Valor por defecto en caso de error general

# Clase RentabilidadCalculator
class RentabilidadCalculator:
    def __init__(self):
        # Datos históricos de rentabilidad por fondo (reales de Chile)
        # Promedio de rentabilidades reales de los últimos 10 años por tipo de fondo
        self.rentabilidad_historica = {
            "A": {
                "1_year": 5.2,  # rentabilidad último año
                "3_year": 5.8,  # promedio últimos 3 años
                "5_year": 6.2,  # promedio últimos 5 años
                "10_year": 7.1  # promedio últimos 10 años
            },
            "B": {
                "1_year": 4.3,
                "3_year": 4.9,
                "5_year": 5.3,
                "10_year": 6.2
            },
            "C": {
                "1_year": 3.6,
                "3_year": 4.1,
                "5_year": 4.5,
                "10_year": 5.4
            },
            "D": {
                "1_year": 2.8,
                "3_year": 3.4,
                "5_year": 3.7,
                "10_year": 4.5
            },
            "E": {
                "1_year": 1.9,
                "3_year": 2.2,
                "5_year": 2.5,
                "10_year": 3.2
            }
        }

        # Volatilidad histórica por fondo (desviación estándar de rentabilidad anual)
        self.volatilidad = {
            "A": 12.5,
            "B": 9.2,
            "C": 7.4,
            "D": 5.1,
            "E": 3.2
        }

        # Inflación promedio anual estimada
        self.inflacion_anual = 3.4  # Inflación promedio de Chile

        # Expectativa de vida después de jubilación (años)
        self.expectativa_vida = {
            "hombre": 85 - 65,  # 65 años edad jubilación hombres, expectativa total 85
            "mujer": 88 - 60    # 60 años edad jubilación mujeres, expectativa total 88
        }

    def calcular_rentabilidad_proyectada(self, fondo, años, escenario="normal"):
        """
        Calcula la rentabilidad proyectada según el fondo, años y escenario
        Escenarios: 'optimista', 'normal', 'pesimista'
        """
        # Obtener rentabilidad base según periodo histórico apropiado
        if años <= 3:
            base = self.rentabilidad_historica[fondo]["1_year"]
        elif años <= 5:
            base = self.rentabilidad_historica[fondo]["3_year"]
        elif años <= 10:
            base = self.rentabilidad_historica[fondo]["5_year"]
        else:
            base = self.rentabilidad_historica[fondo]["10_year"]

        # Ajustar según escenario
        if escenario == "optimista":
            # Escenario optimista: 30% por encima del promedio histórico
            return base * 1.3
        elif escenario == "pesimista":
            # Escenario pesimista: 30% por debajo del promedio histórico
            return base * 0.7
        else:
            # Escenario normal: promedio histórico
            return base

    def calcular_pension_proyectada(self, datos_usuario, escenario="normal"):
        """
        Calcula pensión proyectada basada en datos del usuario

        Args:
            datos_usuario: diccionario con datos del usuario
                - saldo_actual: saldo actual en la AFP
                - edad: edad actual
                - sueldo_mensual: sueldo mensual actual
                - fondo: fondo actual (A, B, C, D, E)
                - años_cotizando: años que lleva cotizando
                - tipo_trabajador: 'Dependiente' o 'Independiente'
                - genero: 'hombre' o 'mujer'
            escenario: 'optimista', 'normal', 'pesimista'

        Returns:
            diccionario con proyecciones
        """
        # Extraer datos necesarios
        saldo_actual = datos_usuario["saldo_actual"]
        edad = datos_usuario["edad"]
        sueldo_mensual = datos_usuario["sueldo_mensual"]
        fondo_actual = datos_usuario["fondo"]
        genero = datos_usuario.get("genero", "hombre").lower()

        # Determinar edad de jubilación según género
        edad_jubilacion = 60 if genero == "mujer" else 65

        # Calcular años hasta jubilación
        años_para_jubilar = max(0, edad_jubilacion - edad)

        # Determinar expectativa de vida tras jubilación
        expectativa_tras_jubilacion = self.expectativa_vida[genero]

        # Cotización mensual (10% obligatorio)
        cotizacion_mensual = sueldo_mensual * 0.10
        cotizacion_anual = cotizacion_mensual * 12

        # Calcular rentabilidad proyectada según escenario
        rentabilidad_anual = self.calcular_rentabilidad_proyectada(
            fondo_actual, años_para_jubilar, escenario) / 100

        # Proyección de saldo a edad de jubilación
        saldo_proyectado = saldo_actual

        # Lista para seguimiento de crecimiento del saldo por año
        saldo_por_año = [saldo_actual]

        # Incremento anual del sueldo (crecimiento real, descontando inflación)
        # Estimamos un 2% de crecimiento real anual del sueldo
        incremento_sueldo_anual = 0.02

        # Simulación año por año
        sueldo_actual = sueldo_mensual
        for año in range(1, años_para_jubilar + 1):
            # Actualizar sueldo con crecimiento anual
            sueldo_actual *= (1 + incremento_sueldo_anual)

            # Actualizar cotización según nuevo sueldo
            cotizacion_anual = (sueldo_actual * 0.10) * 12

            # Actualizar saldo con rentabilidad y cotizaciones
            saldo_proyectado = saldo_proyectado * (1 + rentabilidad_anual) + cotizacion_anual

            # Registrar saldo proyectado para este año
            saldo_por_año.append(saldo_proyectado)

            # Si faltan menos de 10 años para jubilación, simulamos cambio de fondo
            # hacia uno más conservador (seguimos las recomendaciones AFP)
            if años_para_jubilar - año == 10 and fondo_actual == "A":
                fondo_actual = "B"
                rentabilidad_anual = self.calcular_rentabilidad_proyectada(
                    fondo_actual, años_para_jubilar - año, escenario) / 100

            elif años_para_jubilar - año == 5 and fondo_actual in ["A", "B"]:
                fondo_actual = "C"
                rentabilidad_anual = self.calcular_rentabilidad_proyectada(
                    fondo_actual, años_para_jubilar - año, escenario) / 100

            elif años_para_jubilar - año == 3 and fondo_actual in ["A", "B", "C"]:
                fondo_actual = "D"
                rentabilidad_anual = self.calcular_rentabilidad_proyectada(
                    fondo_actual, años_para_jubilar - año, escenario) / 100

        # Cálculo de pensión mensual usando factor actuarial simplificado
        # Basado en expectativa de vida al jubilar y una tasa de descuento

        # Tasa de descuento para valor presente (tasa técnica)
        tasa_descuento = 0.03  # 3% real anual

        # Factor actuarial simplificado (anualidad)
        # Fórmula: (1 - (1 + tasa_descuento)^(-expectativa_vida)) / tasa_descuento
        if tasa_descuento > 0:
            factor_actuarial = (1 - (1 + tasa_descuento)**(-expectativa_tras_jubilacion)) / tasa_descuento
        else:
            factor_actuarial = expectativa_tras_jubilacion

        # Pensión mensual estimada
        pension_mensual = saldo_proyectado / (factor_actuarial * 12)

        # Tasa de reemplazo (% del último sueldo)
        tasa_reemplazo = (pension_mensual / sueldo_actual) * 100

        # Generar visualización
        grafico_b64 = self.generar_grafico_proyeccion(saldo_por_año, años_para_jubilar)

        # Preparar resultados
        resultados = {
            "saldo_actual": saldo_actual,
            "saldo_proyectado": saldo_proyectado,
            "pension_mensual_estimada": pension_mensual,
            "tasa_reemplazo": tasa_reemplazo,
            "años_para_jubilar": años_para_jubilar,
            "ultimo_sueldo_proyectado": sueldo_actual,
            "escenario": escenario,
            "rentabilidad_anual": rentabilidad_anual * 100,  # Convertir a porcentaje
            "grafico_b64": grafico_b64,
            "edad_jubilacion": edad_jubilacion,
            "expectativa_vida_total": edad_jubilacion + expectativa_tras_jubilacion,
            "factor_actuarial": factor_actuarial  # Añadimos el factor actuarial para cálculos posteriores
        }

        return resultados

    def generar_grafico_proyeccion(self, saldo_por_año, años_para_jubilar):
        """
        Genera un gráfico de proyección del saldo en el tiempo
        Retorna el gráfico codificado en base64
        """
        # Crear buffer para la imagen
        buffer = io.BytesIO()

        # Crear figura y ejes
        plt.figure(figsize=(10, 6))
        plt.plot(range(0, años_para_jubilar + 1), saldo_por_año, marker='o', linestyle='-', color='blue')

        # Agregar títulos y etiquetas
        plt.title('Proyección del Saldo AFP a lo Largo del Tiempo', fontsize=15)
        plt.xlabel('Años', fontsize=12)
        plt.ylabel('Saldo (CLP)', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)

        # Formatear eje Y para mostrar cifras en millones
        millones = plt.FuncFormatter(lambda x, pos: f'{x/1000000:.1f}M')
        plt.gca().yaxis.set_major_formatter(millones)

        # Guardar imagen en buffer
        plt.tight_layout()
        plt.savefig(buffer, format='png')
        buffer.seek(0)

        # Convertir imagen a base64
        img_b64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        plt.close()

        return img_b64
        def calcular_multi_escenarios(self, datos_usuario):
          """
        Calcula la pensión bajo múltiples escenarios
        """
        escenarios = {
            "optimista": self.calcular_pension_proyectada(datos_usuario, "optimista"),
            "normal": self.calcular_pension_proyectada(datos_usuario, "normal"),
            "pesimista": self.calcular_pension_proyectada(datos_usuario, "pesimista")
        }

        return escenarios

    def generar_recomendaciones(self, datos_usuario, resultados):
        """
        Genera recomendaciones personalizadas basadas en resultados
        """
        recomendaciones = []

        # Extraer datos
        edad = datos_usuario["edad"]
        tasa_reemplazo = resultados["normal"]["tasa_reemplazo"]
        años_para_jubilar = resultados["normal"]["años_para_jubilar"]
        fondo_actual = datos_usuario["fondo"]

        # Recomendación 1: Tasa de reemplazo
        if tasa_reemplazo < 30:
            recomendaciones.append("Tu tasa de reemplazo proyectada es muy baja. Te recomendamos aumentar tu ahorro previsional voluntario (APV) para mejorar tu futura pensión.")
        elif tasa_reemplazo < 50:
            recomendaciones.append("Tu tasa de reemplazo proyectada es moderada. Considera realizar aportes voluntarios periódicos para aumentar tu pensión futura.")
        else:
            recomendaciones.append("Tu tasa de reemplazo proyectada es buena. Mantén tu estrategia de ahorro previsional.")

        # Recomendación 2: Fondo adecuado según edad
        fondo_recomendado = self._obtener_fondo_recomendado(edad)
        if fondo_actual != fondo_recomendado:
            recomendaciones.append(f"Considerando tu edad ({edad} años), el fondo recomendado es el Fondo {fondo_recomendado}. Actualmente estás en el Fondo {fondo_actual}.")

        # Recomendación 3: APV
        if años_para_jubilar > 0:
            # Calculamos cuánto APV mensual necesitaría para aumentar la tasa de reemplazo en 10%
            sueldo_mensual = datos_usuario["sueldo_mensual"]
            pension_objetivo = (tasa_reemplazo + 10) / 100 * resultados["normal"]["ultimo_sueldo_proyectado"]
            pension_actual = resultados["normal"]["pension_mensual_estimada"]
            diferencia_pension = pension_objetivo - pension_actual

            if diferencia_pension > 0:
                # Cálculo simplificado de APV mensual necesario
                factor_actuarial = resultados["normal"].get("factor_actuarial", 20)  # Valor por defecto si no existe
                apv_mensual_recomendado = (diferencia_pension * 12 * factor_actuarial) / (años_para_jubilar * 12)

                # Ajustar para que sea un valor razonable (entre 5% y 20% del sueldo)
                apv_mensual_recomendado = max(min(apv_mensual_recomendado, sueldo_mensual * 0.2), sueldo_mensual * 0.05)

                recomendaciones.append(f"Para aumentar tu tasa de reemplazo en aproximadamente 10%, te recomendamos un APV mensual de aproximadamente ${int(apv_mensual_recomendado):,} pesos.")

        # Recomendación 4: Educación financiera
        recomendaciones.append("Recuerda revisar periódicamente tus proyecciones de pensión y ajustar tu estrategia de ahorro según cambien tus circunstancias.")

        return recomendaciones

    def _obtener_fondo_recomendado(self, edad):
        """Determina el fondo recomendado según la edad"""
        if edad <= 35:
            return "A"
        elif edad <= 45:
            return "B"
        elif edad <= 55:
            return "C"
        elif edad <= 65:
            return "D"
        else:
            return "E"

    def formatear_resultados(self, datos_usuario, incluir_recomendaciones=True):
        """
        Formatea los resultados para mostrarlos al usuario
        """
        # Calcular múltiples escenarios
        resultados = self.calcular_multi_escenarios(datos_usuario)

        # Obtener resultado del escenario normal para la mayoría de la información
        resultado_normal = resultados["normal"]

        # Formatear respuesta
        respuesta = f"📊 PROYECCIÓN DE PENSIÓN PERSONALIZADA 📊\n\n"

        # Datos básicos
        respuesta += f"👤 Edad actual: {datos_usuario['edad']} años\n"
        respuesta += f"💰 Saldo actual: ${int(datos_usuario['saldo_actual']):,} pesos\n"
        respuesta += f"💼 Sueldo mensual: ${int(datos_usuario['sueldo_mensual']):,} pesos\n"
        respuesta += f"📈 Fondo actual: {datos_usuario['fondo']}\n\n"

        # Resultados de la proyección
        respuesta += f"🔮 RESULTADOS DE LA PROYECCIÓN:\n\n"
        respuesta += f"⏱️ Años para jubilación: {resultado_normal['años_para_jubilar']}\n"
        respuesta += f"🏦 Saldo proyectado al jubilar: ${int(resultado_normal['saldo_proyectado']):,} pesos\n\n"

        # Comparación de escenarios
        respuesta += f"🔄 COMPARATIVA DE ESCENARIOS:\n\n"

        # Tabla de comparación
        respuesta += f"{'Escenario':<12} {'Pensión Mensual':<25} {'Tasa Reemplazo':<18}\n"
        respuesta += f"{'-'*60}\n"

        for escenario, res in resultados.items():
            pension = int(res['pension_mensual_estimada'])
            tasa = res['tasa_reemplazo']
            respuesta += f"{escenario.title():<12} ${pension:,} pesos {' '*5} {tasa:.1f}%\n"

        respuesta += f"\n✨ Escenario normal: Pensión de ${int(resultado_normal['pension_mensual_estimada']):,} pesos mensuales\n"
        respuesta += f"✨ Tasa de reemplazo: {resultado_normal['tasa_reemplazo']:.1f}% de tu último sueldo\n\n"

        # Agregar recomendaciones si se solicitan
        if incluir_recomendaciones:
            recomendaciones = self.generar_recomendaciones(datos_usuario, resultados)

            respuesta += f"🔍 RECOMENDACIONES PERSONALIZADAS:\n\n"
            for i, rec in enumerate(recomendaciones, 1):
                respuesta += f"{i}. {rec}\n"

            respuesta += f"\n"

        # Notas finales
        respuesta += f"📝 NOTAS IMPORTANTES:\n"
        respuesta += f"• Estos cálculos son estimaciones basadas en supuestos y rentabilidades históricas.\n"
        respuesta += f"• Las rentabilidades pasadas no garantizan rentabilidades futuras.\n"
        respuesta += f"• Te recomendamos revisar tu estrategia de ahorro previsional periódicamente.\n"
        respuesta += f"• Para una asesoría más personalizada, contacta a un ejecutivo de AFP Modelo.\n"

        return respuesta

    def calcular_proyecciones_todos_fondos(self, datos_usuario, escenario="normal"):
        """
        Calcula proyecciones para todos los fondos (A, B, C, D, E)

        Args:
            datos_usuario: diccionario con datos del usuario
            escenario: 'optimista', 'normal', 'pesimista'

        Returns:
            diccionario con resultados para cada fondo y el mejor fondo
        """
        resultados_por_fondo = {}
        mejor_pension = 0
        mejor_fondo = None

        # Guardar el fondo original
        fondo_original = datos_usuario["fondo"]

        # Calcular para cada fondo
        for fondo in ["A", "B", "C", "D", "E"]:
            # Modificar temporalmente el fondo para el cálculo
            datos_usuario["fondo"] = fondo

            # Calcular proyección
            resultado = self.calcular_pension_proyectada(datos_usuario, escenario)
            resultados_por_fondo[fondo] = resultado

            # Verificar si es el mejor fondo hasta ahora
            pension_mensual = resultado["pension_mensual_estimada"]
            if pension_mensual > mejor_pension:
                mejor_pension = pension_mensual
                mejor_fondo = fondo

        # Restaurar el fondo original
        datos_usuario["fondo"] = fondo_original

        # Organizar resultados
        return {
            "resultados_por_fondo": resultados_por_fondo,
            "mejor_fondo": mejor_fondo,
            "mejor_pension": mejor_pension
        }


# Función modificada para la comparativa de fondos con integración a la hoja Calculadora
def calcular_comparativa_fondos(db_manager=None, rut_input=None):
    """
    Calcula y compara la pensión proyectada entre los diferentes fondos
    guardando los resultados en la hoja Calculadora

    Args:
        db_manager: instancia de DBManager (opcional)
        rut_input: RUT del usuario (opcional)
    """
    print("\n========== CALCULADORA DE PENSIÓN AFP MODELO ==========")
    print("Comparativa de proyecciones de pensión según diferentes fondos\n")

    # Crear instancia de la calculadora
    calculadora = RentabilidadCalculator()

    # Crear instancia del db_manager si no se proporcionó
    if db_manager is None:
        try:
            db_manager = DBManager('afp-modelo-credentials.json', 'AFP_Modelo_Database')
        except Exception as e:
            print(f"❌ Error al crear el gestor de base de datos: {e}")
            db_manager = None

    # Datos del usuario (desde el sheet o ingresados manualmente)
    datos_usuario = None

    # Si tenemos un RUT y un DB manager, intentar obtener los datos del usuario
    if rut_input and db_manager:
        datos_usuario = db_manager.obtener_datos_usuario(rut_input)
        if datos_usuario:
            print(f"✅ Datos de {datos_usuario['nombre']} cargados exitosamente")

    # Si no se obtuvieron datos, solicitarlos al usuario
    if not datos_usuario:
        print("Ingresa tus datos personales:")

        try:
            rut = rut_input if rut_input else input("RUT (sin puntos, con guión): ")
            nombre = input("Nombre completo: ")
            edad = int(input("Edad actual: "))
            genero = input("Género (hombre/mujer): ").lower()

            # Validar género
            if genero not in ["hombre", "mujer"]:
                print("Género no reconocido. Estableciendo como 'hombre' por defecto.")
                genero = "hombre"

            saldo_actual = float(input("Saldo actual en AFP (pesos, sin puntos ni comas): "))
            sueldo_mensual = float(input("Sueldo mensual (pesos, sin puntos ni comas): "))
            años_cotizando = int(input("Años cotizando: "))
            tipo_trabajador = input("Tipo de trabajador (Dependiente/Independiente): ")
            fondo_actual = input("Fondo actual (A, B, C, D o E): ").upper()

            # Validar fondo
            if fondo_actual not in ["A", "B", "C", "D", "E"]:
                print("Fondo no válido. Usando Fondo C por defecto.")
                fondo_actual = "C"

        except ValueError as e:
            print(f"Error en la entrada de datos: {e}")
            print("Usando valores por defecto para continuar con la simulación.")
            rut = "11111111-1"
            nombre = "Usuario Ejemplo"
            edad = 35
            genero = "hombre"
            saldo_actual = 15000000
            sueldo_mensual = 1000000
            años_cotizando = 10
            tipo_trabajador = "Dependiente"
            fondo_actual = "C"

        # Crear diccionario de datos de usuario
        datos_usuario = {
            "rut": rut,
            "nombre": nombre,
            "edad": edad,
            "genero": genero,
            "saldo_actual": saldo_actual,
            "sueldo_mensual": sueldo_mensual,
            "años_cotizando": años_cotizando,
            "tipo_trabajador": tipo_trabajador,
            "fondo": fondo_actual
        }

    # Calcular para cada fondo
    print("\nCalculando proyecciones para cada fondo...")

    # Obtener proyecciones para todos los fondos
    resultados = calculadora.calcular_proyecciones_todos_fondos(datos_usuario)
    resultados_por_fondo = resultados["resultados_por_fondo"]
    mejor_fondo = resultados["mejor_fondo"]

    # Guardar resultados en la hoja Calculadora (si tenemos DB manager)
    if db_manager:
        guardado = db_manager.guardar_proyecciones_por_fondos_en_calculadora(datos_usuario, resultados_por_fondo)
        if guardado:
            print("✅ Resultados guardados exitosamente en la hoja Calculadora")
        else:
            print("❌ No se pudieron guardar los resultados en la hoja Calculadora")

    # Mostrar resultados comparativos
    print("\n=== RESULTADOS COMPARATIVOS POR FONDO ===\n")

    # Preparar tabla
    print("┌─────────┬────────────────┬─────────────────┬────────────────┬──────────────┐")
    print("│ Fondo   │ Saldo Final    │ Pensión Mensual │ Tasa Reemplazo │ Rentabilidad │")
    print("├─────────┼────────────────┼─────────────────┼────────────────┼──────────────┤")

    for fondo, resultados in sorted(resultados_por_fondo.items()):
        saldo = f"${int(resultados['saldo_proyectado']):,}"
        pension = f"${int(resultados['pension_mensual_estimada']):,}"
        tasa = f"{resultados['tasa_reemplazo']:.1f}%"
        rentabilidad = f"{resultados['rentabilidad_anual']:.2f}%"

        # Marcar el mejor fondo
        es_mejor = "★ " if fondo == mejor_fondo else "  "

        print(f"│ {es_mejor}{fondo:<5} │ {saldo:<14} │ {pension:<15} │ {tasa:<14} │ {rentabilidad:<12} │")

    print("└─────────┴────────────────┴─────────────────┴────────────────┴──────────────┘")

    # Información del mejor fondo
    print(f"\n★ El Fondo {mejor_fondo} te ofrece la mejor pensión estimada: ${int(resultados_por_fondo[mejor_fondo]['pension_mensual_estimada']):,} pesos mensuales")

    # Información adicional sobre el fondo recomendado según edad
    fondo_recomendado = calculadora._obtener_fondo_recomendado(datos_usuario["edad"])
    if fondo_recomendado != mejor_fondo:
        print(f"\nNota: Por tu edad ({datos_usuario['edad']} años), el fondo recomendado es el Fondo {fondo_recomendado}, ")
        print(f"aunque el Fondo {mejor_fondo} proyecta una mejor pensión mensual.")
        print("Ten en cuenta que los fondos con mayor rentabilidad también tienen mayor riesgo y volatilidad.")
    else:
        print(f"\nEl Fondo {mejor_fondo} es el más adecuado para tu perfil de edad y te ofrece la mejor proyección.")

    # Información sobre edad de jubilación
    edad_jubilacion = 60 if datos_usuario["genero"] == "mujer" else 65
    años_para_jubilar = max(0, edad_jubilacion - datos_usuario["edad"])
    print(f"\nTe faltan {años_para_jubilar} años para jubilar (edad de jubilación: {edad_jubilacion} años)")

    # Recomendaciones para mejorar la pensión
    print("\n=== RECOMENDACIONES PARA MEJORAR TU PENSIÓN ===\n")

    # Calcular APV recomendado para mejorar tasa de reemplazo en 10%
    pension_actual = resultados_por_fondo[mejor_fondo]['pension_mensual_estimada']
    ultimo_sueldo = resultados_por_fondo[mejor_fondo]['ultimo_sueldo_proyectado']
    tasa_actual = resultados_por_fondo[mejor_fondo]['tasa_reemplazo']
    pension_objetivo = (tasa_actual + 10) / 100 * ultimo_sueldo
    diferencia_pension = pension_objetivo - pension_actual

    if diferencia_pension > 0 and años_para_jubilar > 0:
        factor_actuarial = resultados_por_fondo[mejor_fondo].get("factor_actuarial", 20)
        apv_mensual = (diferencia_pension * 12 * factor_actuarial) / (años_para_jubilar * 12)
        apv_mensual = max(min(apv_mensual, datos_usuario["sueldo_mensual"] * 0.2), datos_usuario["sueldo_mensual"] * 0.05)

        print(f"1. Para aumentar tu tasa de reemplazo en aproximadamente 10%, ")
        print(f"   te recomendamos un APV mensual de aproximadamente ${int(apv_mensual):,} pesos.")

    # Otras recomendaciones generales
    print("2. Mantén la constancia en tus cotizaciones, evitando lagunas previsionales.")
    print("3. Considera trabajar algunos años adicionales después de la edad legal de jubilación.")
    print("4. Revisa periódicamente tu estrategia de inversión y ajusta según tu perfil de riesgo.")

    # Aclaraciones finales
    print("\n=== NOTAS IMPORTANTES ===\n")
    print("• Estos cálculos son estimaciones basadas en supuestos y rentabilidades históricas de AFP Modelo.")
    print("• Las rentabilidades pasadas no garantizan rentabilidades futuras.")

    # Preguntar si quiere repetir
    repetir = input("\n¿Deseas realizar otra simulación? (s/n): ").lower()
    return repetir == 's'


# Clase del chatbot AFP Modelo
class ChatbotAFPModelo:
    def __init__(self):
        self.nombre = "AsistenteDigital AFP Modelo"

        # Datos de usuarios para pruebas (actualizados con género)
        self.usuarios = {
            "12345678-9": {
                "nombre": "Juan Pérez",
                "saldo": 15678900,
                "fondo": "A",
                "ultima_cotizacion": "15/01/2025",
                "edad": 35,
                "sueldo_mensual": 1200000,
                "años_cotizando": 10,
                "tipo_trabajador": "Dependiente",
                "genero": "hombre"  # Agregamos género para cálculos más precisos
            },
            "98765432-1": {
                "nombre": "María González",
                "saldo": 23456700,
                "fondo": "C",
                "ultima_cotizacion": "20/01/2025",
                "edad": 42,
                "sueldo_mensual": 1500000,
                "años_cotizando": 18,
                "tipo_trabajador": "Independiente",
                "genero": "mujer"
            }
        }

        # Datos de AFP Modelo
        self.afp_modelo = {
            "comision": 0.58,
            "rentabilidades_promedio_anual": {
                "A": 7.89,
                "B": 6.54,
                "C": 5.22,
                "D": 3.98,
                "E": 2.75
            }
        }

        # Inicializar la calculadora de rentabilidad
        self.rentabilidad_calculator = RentabilidadCalculator()

        # Patrones de consultas (actualizados para incluir nuevas opciones)
        self.patrones = [
            (r"hola|buenos días|buenas tardes|saludar", "Saludo"),
            (r"saldo|cuánto tengo|mi dinero|mis fondos", "Saldo"),
            (r"cambiar.*(fondo|tipo)", "Cambio de Fondo"),
            (r"rentabilidad|ganancia|rendimiento", "Rentabilidad"),
            (r"comisión|comisiones|cobro|cobra", "Comisiones"),
            (r"fondos|tipos de fondo|características", "Información Fondos"),
            (r"ayuda|opciones|qué puedes hacer", "Ayuda"),
            (r"menú|menu|opciones|que puedo hacer", "Menú"),
            (r"pensión|pension|calcular", "Calcular Pensión"),
            (r"simulador|simular|proyección|proyectar", "Simulador Pensión"),
            (r"escenarios|optimista|pesimista", "Escenarios Pensión"),
            (r"contacto|ejecutivo|hablar", "Contacto")
        ]

    def determinar_tipo_consulta(self, mensaje):
        """Determina el tipo de consulta según los patrones definidos"""
        mensaje = mensaje.lower()

        for patron, tipo in self.patrones:
            if re.search(patron, mensaje):
                return tipo

        return "General"

    def responder(self, mensaje, rut=None):
        """Genera una respuesta según el tipo de consulta"""
        tipo_consulta = self.determinar_tipo_consulta(mensaje)

        if tipo_consulta == "Saludo":
            return self._saludar(rut)
        elif tipo_consulta == "Saldo":
            return self._consultar_saldo(rut)
        elif tipo_consulta == "Rentabilidad":
            return self._consultar_rentabilidad()
        elif tipo_consulta == "Cambio de Fondo":
            return self._info_cambio_fondo(rut)
        elif tipo_consulta == "Comisiones":
            return self._consultar_comisiones()
        elif tipo_consulta == "Información Fondos":
            return self._info_fondos()
        elif tipo_consulta == "Ayuda" or tipo_consulta == "Menú":
            return self._mostrar_ayuda()
        elif tipo_consulta == "Calcular Pensión" or tipo_consulta == "Simulador Pensión":
            return self._calcular_pension_avanzada(rut)
        elif tipo_consulta == "Escenarios Pensión":
            return self._mostrar_escenarios_pension(rut)
        elif tipo_consulta == "Contacto":
            return "Para contactar con un ejecutivo, puedes llamar al 600 828 7200 o enviar un email a contacto@afpmodelo.cl"
        else:
            return self._respuesta_generica()

    def _saludar(self, rut=None):
        if rut and rut in self.usuarios:
            return f"¡Hola {self.usuarios[rut]['nombre']}! ¿En qué puedo ayudarte hoy?\n\nPuedes escribir 'ayuda' o '9' para ver el menú de opciones."
        else:
            return "¡Hola! Soy el asistente virtual de AFP Modelo. ¿En qué puedo ayudarte hoy?\n\nPuedes escribir 'ayuda' o '9' para ver el menú de opciones."

    def _consultar_saldo(self, rut=None):
        if not rut or rut not in self.usuarios:
            return "Para consultar tu saldo, necesito que me proporciones tu RUT. Por seguridad, te recomiendo hacerlo a través de nuestra app o sitio web oficial."

        usuario = self.usuarios[rut]
        return f"Tu saldo actual en el Fondo {usuario['fondo']} es de ${usuario['saldo']:,} pesos. Tu última cotización fue registrada el {usuario['ultima_cotizacion']}."

    def _consultar_rentabilidad(self):
        respuesta = "Las rentabilidades anuales promedio de AFP Modelo son:\n"
        for fondo, rentabilidad in self.afp_modelo["rentabilidades_promedio_anual"].items():
            respuesta += f"- Fondo {fondo}: {rentabilidad}%\n"
        return respuesta

    def _info_cambio_fondo(self, rut=None):
        respuesta = "En AFP Modelo puedes cambiar de fondo fácilmente a través de nuestra web o app.\n\n"
        respuesta += "Los fondos disponibles son:\n"
        respuesta += "- Fondo A: Mayor riesgo, mayor rentabilidad potencial (recomendado para menores de 35 años)\n"
        respuesta += "- Fondo B: Riesgo alto (recomendado para personas entre 36-45 años)\n"
        respuesta += "- Fondo C: Riesgo moderado (recomendado para personas entre 46-55 años)\n"
        respuesta += "- Fondo D: Bajo riesgo (recomendado para personas entre 56-65 años)\n"
        respuesta += "- Fondo E: Muy bajo riesgo (recomendado para mayores de 65 años)\n"

        if rut and rut in self.usuarios:
            edad = self.usuarios[rut]["edad"]
            fondo_recomendado = self.rentabilidad_calculator._obtener_fondo_recomendado(edad)
            respuesta += f"\nPor tu edad ({edad} años), el fondo recomendado para ti es el Fondo {fondo_recomendado}."

        return respuesta

    def _consultar_comisiones(self):
        return f"AFP Modelo tiene la comisión más baja del mercado: {self.afp_modelo['comision']}% sobre el sueldo imponible, sin comisión fija adicional."

    def _info_fondos(self):
        return """AFP Modelo ofrece 5 fondos con distintos niveles de riesgo:
- Fondo A: Mayor riesgo (80% renta variable), mayor rentabilidad potencial
- Fondo B: Riesgo alto (60% renta variable)
- Fondo C: Riesgo moderado (40% renta variable)
- Fondo D: Bajo riesgo (20% renta variable)
- Fondo E: Muy bajo riesgo (5% renta variable), menor rentabilidad"""

    def _mostrar_menu_interactivo(self):
        """Muestra un menú interactivo con todas las opciones disponibles (actualizado)"""
        menu = """
╔════════════════════════════════════════════════╗
║           MENÚ DE ASISTENTE AFP MODELO         ║
╠════════════════════════════════════════════════╣
║ 1. Consultar mi saldo                          ║
║ 2. Ver rentabilidades de fondos                ║
║ 3. Información sobre comisiones                ║
║ 4. Características de los fondos               ║
║ 5. Cambiar de fondo                            ║
║ 6. Calcular pensión estimada                   ║
║ 7. Simular escenarios de pensión               ║
║ 8. Contactar con ejecutivo                     ║
║ 9. Volver al menú principal                    ║
╚════════════════════════════════════════════════╝
"""
        return menu

    def _mostrar_ayuda(self):
        respuesta = "Estas son las opciones con las que puedo ayudarte:\n"
        respuesta += self._mostrar_menu_interactivo()
        respuesta += "\nPuedes seleccionar una opción escribiendo su número o preguntándome directamente."
        return respuesta

    def _respuesta_generica(self):
        respuesta = "No he podido entender tu consulta. Aquí tienes las opciones disponibles:\n"
        respuesta += self._mostrar_menu_interactivo()
        respuesta += "\nPor favor, escribe el número o nombre de la opción que deseas consultar."
        return respuesta

    def _calcular_pension_avanzada(self, rut=None):
        """Usa la calculadora avanzada de rentabilidad para proyectar la pensión"""
        if not rut or rut not in self.usuarios:
            return "Para calcular tu pensión estimada necesito que me proporciones tu RUT y estés registrado."

        usuario = self.usuarios[rut]

        # Formatear datos para la calculadora
        datos_usuario = {
            "saldo_actual": usuario["saldo"],
            "edad": usuario["edad"],
            "sueldo_mensual": usuario["sueldo_mensual"],
            "fondo": usuario["fondo"],
            "años_cotizando": usuario["años_cotizando"],
            "tipo_trabajador": usuario["tipo_trabajador"],
            "genero": usuario.get("genero", "hombre")
        }

        # Obtener resultados formateados
        respuesta = self.rentabilidad_calculator.formatear_resultados(datos_usuario)

        # Agregar llamado a la acción
        respuesta += "\n\nSi deseas ver una comparación detallada de diferentes escenarios, escribe 'escenarios de pensión'."

        return respuesta

    def _mostrar_escenarios_pension(self, rut=None):
        """Muestra una comparación de diferentes escenarios de pensión"""
        if not rut or rut not in self.usuarios:
            return "Para simular escenarios de pensión necesito que me proporciones tu RUT y estés registrado."

        usuario = self.usuarios[rut]

        # Formatear datos para la calculadora
        datos_usuario = {
            "saldo_actual": usuario["saldo"],
            "edad": usuario["edad"],
            "sueldo_mensual": usuario["sueldo_mensual"],
            "fondo": usuario["fondo"],
            "años_cotizando": usuario["años_cotizando"],
            "tipo_trabajador": usuario["tipo_trabajador"],
            "genero": usuario.get("genero", "hombre")
        }

        # Calcular escenarios
        escenarios = self.rentabilidad_calculator.calcular_multi_escenarios(datos_usuario)

        # Preparar respuesta
        respuesta = f"📊 COMPARATIVA DE ESCENARIOS DE PENSIÓN PARA {usuario['nombre'].upper()} 📊\n\n"

        # Años para jubilación
        años_para_jubilar = escenarios["normal"]["años_para_jubilar"]
        respuesta += f"⏱️ Años para jubilación: {años_para_jubilar}\n\n"

        # Tabla comparativa
        respuesta += "┌─────────────┬────────────────┬─────────────────┬────────────────┐\n"
        respuesta += "│ Escenario   │ Saldo Final    │ Pensión Mensual │ Tasa Reemplazo │\n"
        respuesta += "├─────────────┼────────────────┼─────────────────┼────────────────┤\n"

        for nombre, escenario in escenarios.items():
            saldo = f"${int(escenario['saldo_proyectado']):,}"
            pension = f"${int(escenario['pension_mensual_estimada']):,}"
            tasa = f"{escenario['tasa_reemplazo']:.1f}%"

            respuesta += f"│ {nombre.title():<11} │ {saldo:<14} │ {pension:<15} │ {tasa:<14} │\n"

        respuesta += "└─────────────┴────────────────┴─────────────────┴────────────────┘\n\n"

        # Explicación de escenarios
        respuesta += "📝 EXPLICACIÓN DE ESCENARIOS:\n\n"
        respuesta += "• Escenario Optimista: Considera rentabilidades 30% superiores al promedio histórico.\n"
        respuesta += "• Escenario Normal: Basado en rentabilidades promedio históricas del sistema AFP.\n"
        respuesta += "• Escenario Pesimista: Considera rentabilidades 30% inferiores al promedio histórico.\n\n"

        # Recomendaciones
        recomendaciones = self.rentabilidad_calculator.generar_recomendaciones(datos_usuario, escenarios)

        respuesta += "🔍 RECOMENDACIONES PERSONALIZADAS:\n\n"
        for i, rec in enumerate(recomendaciones, 1):
            respuesta += f"{i}. {rec}\n"

        return respuesta

    # Reemplazar el método de cálculo de pensión simple por el avanzado
    def _calcular_pension_estimada(self, rut=None):
        """Redirecciona al método avanzado de cálculo de pensión"""
        return self._calcular_pension_avanzada(rut)

    def procesar_seleccion_menu(self, seleccion, rut=None):
        """Procesa la selección del menú y retorna la respuesta adecuada (actualizado)"""
        try:
            # Intentar convertir a número si es numérico
            if seleccion.isdigit():
                opcion = int(seleccion)

                if opcion == 1:
                    return self._consultar_saldo(rut)
                elif opcion == 2:
                    return self._consultar_rentabilidad()
                elif opcion == 3:
                    return self._consultar_comisiones()
                elif opcion == 4:
                    return self._info_fondos()
                elif opcion == 5:
                    return self._info_cambio_fondo(rut)
                elif opcion == 6:
                    return self._calcular_pension_avanzada(rut)
                elif opcion == 7:
                    return self._mostrar_escenarios_pension(rut)
                elif opcion == 8:
                    return "Para contactar con un ejecutivo, puedes llamar al 600 828 7200 o enviar un email a contacto@afpmodelo.cl"
                elif opcion == 9:
                    return "VOLVER_MENU_PRINCIPAL"  # Señal especial para volver al menú principal
                else:
                    return "Opción no válida. " + self._mostrar_ayuda()
            else:
                # Si no es un número, procesar como mensaje normal
                return self.responder(seleccion, rut)
        except Exception as e:
            return f"Error al procesar selección: {e}. " + self._mostrar_ayuda()


# Clase que integra el chatbot con la base de datos
class ChatbotAFPModeloConDB:
    def __init__(self, credentials_file='afp-modelo-credentials.json', spreadsheet_name='AFP_Modelo_Database'):
        # Inicializar chatbot
        self.chatbot = ChatbotAFPModelo()

        # Inicializar gestor de base de datos
        self.db_manager = DBManager(credentials_file, spreadsheet_name)

        # Tiempo de inicio de la conversación
        self.start_time = datetime.now()

    def registrar_usuario(self, datos_usuario):
        """Registra un nuevo usuario y lo añade a la memoria local (actualizado para género)"""
        # Registrar en la base de datos
        resultado = self.db_manager.registrar_usuario(datos_usuario)

        if resultado:
            # Añadir a la memoria local
            rut = datos_usuario.get("rut", "")
            if rut:
                # Determinar género basado en nombre (simplificado)
                nombre = datos_usuario.get("nombre", "")
                genero = datos_usuario.get("genero", "")
                if not genero:
                    genero = "mujer" if nombre.split()[-1].lower().endswith(('a', 'z')) else "hombre"

                self.chatbot.usuarios[rut] = {
                    "nombre": datos_usuario.get("nombre", ""),
                    "saldo": 10000000,  # Saldo por defecto
                    "fondo": "C",  # Fondo por defecto
                    "ultima_cotizacion": datetime.now().strftime("%d/%m/%Y"),
                    "edad": self._calcular_edad(datos_usuario.get("fecha_nacimiento", "")),
                    "sueldo_mensual": int(datos_usuario.get("sueldo_mensual", 0)),
                    "años_cotizando": 0,
                    "tipo_trabajador": datos_usuario.get("tipo_trabajador", "Dependiente"),
                    "genero": genero  # Agregamos género
                }
                print(f"¡Usuario {datos_usuario.get('nombre')} registrado correctamente!")
                return True

        return False

    def procesar_mensaje(self, mensaje, rut=None):
        """Procesa un mensaje y registra la interacción"""
        # Verificar si es una selección del menú
        es_seleccion_menu = mensaje.strip().isdigit() and 1 <= int(mensaje) <= 9

        # Obtener respuesta del chatbot
        if es_seleccion_menu:
            respuesta = self.chatbot.procesar_seleccion_menu(mensaje, rut)
            # Verificar si la respuesta es la señal para volver al menú principal
            if respuesta == "VOLVER_MENU_PRINCIPAL":
                return respuesta
        else:
            respuesta = self.chatbot.responder(mensaje, rut)

        # Determinar tipo de consulta
        tipo_consulta = "Menú" if es_seleccion_menu else self.chatbot.determinar_tipo_consulta(mensaje)

        # Registrar interacción
        self.db_manager.registrar_interaccion({
            "rut": rut,
            "tipo_consulta": tipo_consulta,
            "mensaje": mensaje,
            "respuesta": respuesta,
            "canal": "Google Colab",
            "duracion": (datetime.now() - self.start_time).total_seconds()
        })

        # Actualizar tiempo para siguiente interacción
        self.start_time = datetime.now()

        return respuesta

    def _calcular_edad(self, fecha_nacimiento):
        """Calcula edad a partir de fecha de nacimiento"""
        if not fecha_nacimiento:
            return 30  # Valor por defecto

        try:
            fecha_nac = datetime.strptime(fecha_nacimiento, "%Y-%m-%d")
            hoy = datetime.now()
            return hoy.year - fecha_nac.year - ((hoy.month, hoy.day) < (fecha_nac.month, fecha_nac.day))
        except:
            return 30  # Valor por defecto en caso de error


# Función para ejecutar el chatbot modificada para volver al menú principal
def ejecutar_chatbot():
    """
    Ejecuta el chatbot AFP Modelo

    Returns:
        bool: True si se debe volver al menú principal, False si se sale completamente
    """
    print("\n========== CHATBOT AFP MODELO ==========")
    print("Tu asistente virtual para consultas sobre tu fondo de pensiones")
    print("Este chatbot está integrado con Google Sheets para guardar todas las interacciones\n")

    # Crear instancia del chatbot
    chatbot = ChatbotAFPModeloConDB()

    # Simulación de sesión
    rut_input = input("Ingresa tu RUT para identificarte (o presiona Enter para continuar como invitado): ")

    rut = None
    if rut_input in chatbot.chatbot.usuarios:
        rut = rut_input
        print(f"¡Bienvenido/a {chatbot.chatbot.usuarios[rut]['nombre']}!")
    elif rut_input:
        print("RUT no encontrado. ¿Deseas registrarte como nuevo usuario?")
        registrar = input("¿Registrar nuevo usuario? (s/n): ").lower()

        if registrar == 's':
            # Recopilar datos para registro
            nombre = input("Nombre completo: ")
            email = input("Correo electrónico: ")
            telefono = input("Teléfono: ")
            edad = int(input("Edad: "))
            tipo_trabajador = input("Tipo de trabajador (Dependiente/Independiente): ")
            sueldo = input("Sueldo mensual: ")
            genero = input("Género (hombre/mujer): ").lower()

            # Validar género
            if genero not in ["hombre", "mujer"]:
                genero = "hombre" if nombre.split()[-1].lower().endswith(('o', 'e', 'n', 'r', 'l')) else "mujer"
                print(f"Género inferido: {genero}")

            # Cálculo más seguro de la fecha de nacimiento
            año_actual = datetime.now().year
            año_nacimiento = año_actual - edad
            # Usar el primer día del año para simplificar
            fecha_nacimiento = f"{año_nacimiento}-01-01"

            # Registrar usuario
            chatbot.registrar_usuario({
                "rut": rut_input,
                "nombre": nombre,
                "email": email,
                "telefono": telefono,
                "fecha_nacimiento": fecha_nacimiento,
                "tipo_trabajador": tipo_trabajador,
                "sueldo_mensual": sueldo,
                "genero": genero
            })

            rut = rut_input
        else:
            print("Continuando como invitado.")

    # Bucle de chat
    print("\nHola, soy el asistente virtual de AFP Modelo. ¿En qué puedo ayudarte?")
    print("(Escribe 'ayuda' para ver el menú de opciones o 'salir' para terminar)")

    # Mostrar menú inicial para guiar al usuario
    print("\nAsistente AFP Modelo: " + chatbot.chatbot._mostrar_ayuda())

    while True:
        try:
            mensaje = input("\nTú: ")

            if mensaje.lower() in ["salir", "exit", "terminar"]:
                print("\nAsistente AFP Modelo: ¡Gracias por utilizar nuestro servicio! ¡Hasta pronto!")
                return False  # Indicar que se está saliendo completamente

            respuesta = chatbot.procesar_mensaje(mensaje, rut)

            # Verificar si la respuesta es la señal para volver al menú principal
            if respuesta == "VOLVER_MENU_PRINCIPAL":
                print("\nAsistente AFP Modelo: Volviendo al menú principal...")
                return True  # Indicar que se quiere volver al menú principal

            print(f"\nAsistente AFP Modelo: {respuesta}")

            # Si el usuario está confundido, sugerir el menú
            if "no he podido entender" in respuesta.lower() or "no comprendo" in respuesta.lower():
                print("\nPuedes escribir 'ayuda' o '9' para ver todas las opciones disponibles.")

            # Registrar éxito de la interacción
            print(f"Interacción registrada correctamente para {rut or 'invitado'}")

        except Exception as e:
            print(f"❌ Error: {e}")
            print("Escribe 'ayuda' para ver el menú de opciones.")


# Función principal
def main():
    """Función principal para ejecutar la calculadora de comparación de fondos"""
    print("\n========== CALCULADORA AFP MODELO ==========")
    print("Tu herramienta para proyectar tu pensión según diferentes fondos\n")

    # Intentar crear instancia del DB manager una sola vez
    try:
        db_manager = DBManager('afp-modelo-credentials.json', 'AFP_Modelo_Database')
    except Exception as e:
        print(f"❌ Error al crear el gestor de base de datos: {e}")
        db_manager = None

    while True:
        print("\nSelecciona una opción:")
        print("1. Calcular comparativa de pensión por fondos")
        print("2. Iniciar chatbot AFP Modelo")
        print("3. Salir")

        opcion = input("\nOpción: ")

        if opcion == "1":
            repetir = True
            while repetir:
                rut_input = input("\nIngresa tu RUT para cargar tus datos (o Enter para continuar sin RUT): ")
                repetir = calcular_comparativa_fondos(db_manager, rut_input if rut_input else None)
        elif opcion == "2":
            # Ejecutar chatbot y verificar si se quiere volver al menú principal
            volver_al_menu = ejecutar_chatbot()
            if not volver_al_menu:
                # Si el usuario quiere salir completamente
                print("\n¡Gracias por utilizar la Calculadora AFP Modelo! ¡Hasta pronto!")
                break
            # Si volver_al_menu es True, simplemente continúa el bucle y muestra el menú principal
        elif opcion == "3":
            print("\n¡Gracias por utilizar la Calculadora AFP Modelo! ¡Hasta pronto!")
            break
        else:
            print("Opción no válida. Por favor, selecciona una opción correcta.")


# Ejecutar el programa si es el script principal
if __name__ == "__main__":
    main()


========== CALCULADORA AFP MODELO ==========
Tu herramienta para proyectar tu pensión según diferentes fondos

✅ Conectado a 'AFP_Modelo_Database'

Selecciona una opción:
1. Calcular comparativa de pensión por fondos
2. Iniciar chatbot AFP Modelo
3. Salir

Opción: 2

========== CHATBOT AFP MODELO ==========
Tu asistente virtual para consultas sobre tu fondo de pensiones
Este chatbot está integrado con Google Sheets para guardar todas las interacciones

✅ Conectado a 'AFP_Modelo_Database'
Ingresa tu RUT para identificarte (o presiona Enter para continuar como invitado): 19199-1
RUT no encontrado. ¿Deseas registrarte como nuevo usuario?
¿Registrar nuevo usuario? (s/n): s
Nombre completo: jose valverde
Correo electrónico: valverde@gmail.com
Teléfono: 98765431
Edad: 42
Tipo de trabajador (Dependiente/Independiente): dependiente
Sueldo mensual: 1200000
Género (hombre/mujer): mujer
✅ Nuevo usuario 19199-1 registrado
¡Usuario jose valverde registrado correctamente!

Hola, soy el asistente vi